In [1]:
# pip install yfinance

## working file code

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

symbol_df = pd.read_csv('https://raw.githubusercontent.com/Keshkarki/rough-file/master/symbol.csv',header=None).squeeze()
equity_details = pd.read_csv('EQUITY_L.csv')['SYMBOL']
equity_50 = equity_details[equity_details.isin(symbol_df)]

In [3]:
len(equity_50)

50

## downloading csv files single at a time

In [4]:
# for name in equity_50[ :2 ]:
#     data = yf.download(f"{name}.NS", start='2012-01-01', end='2023-05-01')

#     data['Symbol'] = name    # add new column with symbol
#     data.to_csv(f"C:\\Users\\Admin\\OneDrive\\Desktop\\Nifty50_data\\Data3\\{name}.NS.csv")

## PART-1 combine all files

### demo file downloading tesla

In [5]:
# import datetime as dt
# start_date = pd.Timestamp.now() - pd.Timedelta(days=365*10)
# end_date = pd.Timestamp.now()

# df = yf.download(tickers='TSLA',start=start_date,end=end_date)

### working files

In [6]:
data_list = []

start_date = pd.Timestamp.now() - pd.Timedelta(days=365*10)
end_date = pd.Timestamp.now()

for name in equity_50:
    data = yf.download(tickers=F"{name}.NS",start=start_date, end=end_date)
    data['Symbol'] = name
    data_list.append(data)

combined_data = pd.concat(data_list)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
combined_data

,Open,High,Low,Close,Adj Close,Volume,Symbol
Date,,,,,,,
2013-05-10,35.176624,35.291580,34.486889,34.747456,30.428123,4858311,ADANIENT
2013-05-13,34.548199,34.632500,32.946472,33.291344,29.153019,5525517,ADANIENT
2013-05-14,33.230034,33.291344,32.570950,32.915817,28.824177,5024424,ADANIENT
2013-05-15,32.954136,35.008022,32.915817,34.647827,30.340879,6926560,ADANIENT
2013-05-16,34.824093,35.559814,34.502216,35.122978,30.756969,6178676,ADANIENT
...,...,...,...,...,...,...,...
2023-04-28,380.000000,388.000000,380.000000,385.000000,385.000000,13854703,WIPRO
2023-05-02,389.000000,392.149994,387.450012,388.350006,388.350006,4873304,WIPRO
2023-05-03,388.350006,388.350006,382.049988,385.100006,385.100006,2414244,WIPRO


### cal daily return

In [8]:
combined_data['daily_return'] = (((combined_data['Adj Close'] - combined_data['Adj Close'].shift(1)) / combined_data['Adj Close'].shift(1))*100).fillna(method='bfill')
combined_data.insert(0,'day',combined_data.index.day.to_list())

In [9]:
# Resample daily data to monthly data
monthly_data = combined_data.resample('BM').last()

# Calculate monthly returns
monthly_data['monthly_return'] = ((monthly_data['Adj Close'] - monthly_data['Adj Close'].shift(1)) / monthly_data['Adj Close'].shift(1))*100

# Drop the first row as it will have NaN value
monthly_data = monthly_data.dropna()

# Print the monthly returns
print(monthly_data['monthly_return'])

Date
2013-06-28       8.068358
2013-07-31      25.501438
2013-08-30      10.410943
2013-09-30      -2.181550
2013-10-31       0.982983
                 ...     
2023-01-31       1.803286
2023-02-28      -2.958510
2023-03-31      -5.632344
2023-04-28       5.407255
2023-05-31    5632.091112
Freq: BM, Name: monthly_return, Length: 120, dtype: float64


In [10]:
merged_data = combined_data.reset_index()
monthly_data = monthly_data.reset_index()


merged_data = combined_data.merge(monthly_data[['Date', 'monthly_return']], how='left', on='Date')
merged_data['monthly_return'] = merged_data['monthly_return'].fillna(0)
merged_data = merged_data.set_index('Date')

In [11]:
merged_data.iloc[60:60*2]

,day,Open,High,Low,Close,Adj Close,Volume,Symbol,daily_return,monthly_return
Date,,,,,,,,,,
2013-08-02,2,26.976410,27.428572,22.301521,24.186804,22.304230,34902456,ADANIENT,-10.594908,0.000000
2013-08-05,5,24.983833,26.056759,23.849600,25.719555,23.717678,11938093,ADANIENT,6.337131,0.000000
2013-08-06,6,25.443659,25.512632,24.148485,25.152437,23.194702,9566905,ADANIENT,-2.205005,0.000000
2013-08-07,7,24.830559,26.363310,24.569990,25.619925,23.625803,10953256,ADANIENT,1.858618,0.000000
2013-08-08,8,25.596933,26.669861,24.930187,25.504969,23.519796,20369250,ADANIENT,-0.448690,0.000000
2013-08-12,12,25.596933,26.194708,25.290384,25.941803,23.922630,19867217,ADANIENT,1.712744,0.000000
2013-08-13,13,25.750210,26.501257,25.321039,26.233027,24.191185,10418909,ADANIENT,1.122597,0.000000
2013-08-14,14,25.857502,26.654531,24.922523,25.872829,23.859026,10171948,ADANIENT,-1.373058,0.000000
2013-08-16,16,25.443659,25.627588,24.485689,24.547001,22.636391,8454363,ADANIENT,-5.124414,0.000000


In [12]:
# merged_data.to_csv('Combined_data3.csv')

## PART2 - downloading index data

In [13]:
start_date = pd.Timestamp.now() - pd.Timedelta(days=365*10)
end_date = pd.Timestamp.now()

In [14]:
import numpy as np
import pandas as pd
import yfinance as yf

start_date = pd.Timestamp.now() - pd.Timedelta(days=365*10)
end_date = pd.Timestamp.now()


index_df = yf.download('^NSEI',start=start_date, end=end_date)
index_df['Symbol'] = 'index'

index_df['daily_return'] = (((index_df['Adj Close'] - index_df['Adj Close'].shift(1))/index_df['Adj Close'].shift(1))*100).fillna(method='bfill')
index_df.insert(0,'day',index_df.index.day.to_list())
index_df

[*********************100%***********************]  1 of 1 completed


,day,Open,High,Low,Close,Adj Close,Volume,Symbol,daily_return
Date,,,,,,,,,
2013-05-10,10,6046.250000,6105.299805,6045.600098,6094.750000,6094.750000,111400,index,-1.875381
2013-05-13,13,6098.200195,6104.950195,5972.899902,5980.450195,5980.450195,107900,index,-1.875381
2013-05-14,14,5989.700195,6026.200195,5970.049805,5995.399902,5995.399902,119300,index,0.249976
2013-05-15,15,6018.850098,6157.100098,6018.850098,6146.750000,6146.750000,153900,index,2.524437
2013-05-16,16,6128.250000,6187.299805,6128.250000,6169.899902,6169.899902,158600,index,0.376620
...,...,...,...,...,...,...,...,...,...
2023-05-02,2,18124.800781,18180.250000,18101.750000,18147.650391,18147.650391,265200,index,0.457517
2023-05-03,3,18113.800781,18116.349609,18042.400391,18089.849609,18089.849609,226200,index,-0.318503
2023-05-04,4,18081.000000,18267.449219,18066.699219,18255.800781,18255.800781,226600,index,0.917372


In [15]:
# Resample daily data to monthly data
monthly_data2 = index_df.resample('BM').last()

# Calculate monthly returns
monthly_data2['monthly_return'] = ((monthly_data2['Adj Close'] - monthly_data2['Adj Close'].shift(1)) / monthly_data2['Adj Close'].shift(1))*100

# Drop the first row as it will have NaN value
monthly_data2 = monthly_data2.dropna()

# Print the monthly returns
print(monthly_data2['monthly_return'])

Date
2013-06-28   -2.401457
2013-07-31   -1.715111
2013-08-30   -4.705681
2013-09-30    4.815600
2013-10-31    9.831223
                ...   
2023-01-31   -2.447628
2023-02-28   -2.028072
2023-03-31    0.322474
2023-04-28    4.062559
2023-05-31    0.722391
Freq: BM, Name: monthly_return, Length: 120, dtype: float64


In [16]:
index_df = index_df.reset_index()
monthly_data2 = monthly_data2.reset_index()

index_df = index_df.merge(monthly_data2[['Date', 'monthly_return']], how='left', on='Date')
index_df['monthly_return'] = index_df['monthly_return'].fillna(0)

index_df = index_df.set_index('Date')

In [17]:
index_df.iloc[50:100]

,day,Open,High,Low,Close,Adj Close,Volume,Symbol,daily_return,monthly_return
Date,,,,,,,,,,
2013-07-19,19,6057.200195,6066.850098,6020.250000,6029.200195,6029.200195,163300,index,-0.146564,0.000000
2013-07-22,22,6009.750000,6064.149902,6004.250000,6031.799805,6031.799805,115200,index,0.043117,0.000000
2013-07-23,23,6064.299805,6093.350098,6061.299805,6077.799805,6077.799805,126500,index,0.762625,0.000000
2013-07-24,24,6032.200195,6047.250000,5962.600098,5990.500000,5990.500000,151900,index,-1.436372,0.000000
2013-07-25,25,5970.399902,5990.649902,5896.399902,5907.500000,5907.500000,237100,index,-1.385527,0.000000
2013-07-26,26,5937.950195,5944.500000,5869.500000,5886.200195,5886.200195,174500,index,-0.360555,0.000000
2013-07-29,29,5869.950195,5886.000000,5825.799805,5831.649902,5831.649902,173400,index,-0.926749,0.000000
2013-07-30,30,5836.049805,5861.299805,5747.600098,5755.049805,5755.049805,181500,index,-1.313524,0.000000
2013-07-31,31,5738.350098,5752.100098,5675.750000,5742.000000,5742.000000,235500,index,-0.226754,-1.715111


## PART4- merging stock_df with index_df

In [18]:
merged_data = merged_data.reset_index()
index_df = index_df.reset_index()

merged_df = merged_data.merge(index_df[['Symbol', 'Date', 'daily_return', 'monthly_return']], on=['Date'], how='left')
merged_df.fillna({'monthly_return': 0})

merged_df = merged_df.set_index('Date').iloc[1:]

In [19]:
merged_df.head(50)

,day,Open,High,Low,Close,Adj Close,Volume,Symbol_x,daily_return_x,monthly_return_x,Symbol_y,daily_return_y,monthly_return_y
Date,,,,,,,,,,,,,
2013-05-13,13,34.548199,34.632500,32.946472,33.291344,29.153019,5525517,ADANIENT,-4.190546,0.000000,index,-1.875381,0.000000
2013-05-14,14,33.230034,33.291344,32.570950,32.915817,28.824177,5024424,ADANIENT,-1.127987,0.000000,index,0.249976,0.000000
2013-05-15,15,32.954136,35.008022,32.915817,34.647827,30.340879,6926560,ADANIENT,5.261911,0.000000,index,2.524437,0.000000
2013-05-16,16,34.824093,35.559814,34.502216,35.122978,30.756969,6178676,ADANIENT,1.371384,0.000000,index,0.376620,0.000000
2013-05-17,17,35.122978,36.540775,34.640163,36.073284,31.589146,6804198,ADANIENT,2.705651,0.000000,index,0.282013,0.000000
2013-05-20,20,36.632740,38.510357,36.019638,37.230511,32.602520,28470509,ADANIENT,3.207983,0.000000,index,-0.491327,0.000000
2013-05-21,21,37.307148,37.866604,36.249550,36.747696,32.179722,13781602,ADANIENT,-1.296827,0.000000,index,-0.695152,0.000000
2013-05-22,22,36.824333,37.146210,35.444859,35.682434,31.246876,7150341,ADANIENT,-2.898863,0.000000,index,-0.320572,0.000000
2013-05-23,23,35.406536,35.460182,32.892826,33.574902,29.401331,10438534,ADANIENT,-5.906334,0.000000,index,-2.091233,0.000000


In [20]:
merged_df.head(2)

,day,Open,High,Low,Close,Adj Close,Volume,Symbol_x,daily_return_x,monthly_return_x,Symbol_y,daily_return_y,monthly_return_y
Date,,,,,,,,,,,,,
2013-05-13,13,34.548199,34.632500,32.946472,33.291344,29.153019,5525517,ADANIENT,-4.190546,0.0,index,-1.875381,0.0
2013-05-14,14,33.230034,33.291344,32.570950,32.915817,28.824177,5024424,ADANIENT,-1.127987,0.0,index,0.249976,0.0


In [21]:
merged_df.rename(columns={
    'Symbol_x' : 'symbol_stock',
    'daily_return_x' : 'daily_return_stock '	,
    'monthly_return_x' : 'monthly_return_stock',
    'Symbol_y'  : 'Symbol_index',
    'daily_return_y' : "daily_return_index",
    'monthly_return_y' : 'daily_return_index'
},inplace=True)

In [22]:
merged_df.to_csv('merged_file.csv')